## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from google.colab import files # notebook run in colab

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []

for c in list(application_df['CLASSIFICATION'].unique()):
    if application_df['CLASSIFICATION'].value_counts()[c] < 1500:
        classifications_to_replace.append(c)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_cat = pd.get_dummies(application_df[['APPLICATION_TYPE', 'AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']])

# Adding in ASK_AMT column to dummies DF
application_df_cat['ASK_AMT'] = application_df['ASK_AMT']

application_df_cat

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,5000
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,108590
2,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,1,0,5000
3,0,0,0,1,0,0,0,0,0,1,...,1,0,0,0,0,0,0,1,0,6692
4,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,142590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,5000
34295,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,5000
34296,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,5000
34297,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,5000


In [9]:
# Split our preprocessed data into our features and target arrays
X = application_df_cat
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

X_train.to_csv('xtrain.csv')

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [33]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=200, activation='relu', input_dim=36))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=150, activation='relu'))

# Dropout layer - to prevent overfitting
nn.add(tf.keras.layers.Dropout(0.1))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=150, activation='relu'))

# Dropout layer - to prevent overfitting
nn.add(tf.keras.layers.Dropout(0.1))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation='relu'))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 200)               7400      
                                                                 
 dense_41 (Dense)            (None, 150)               30150     
                                                                 
 dropout_16 (Dropout)        (None, 150)               0         
                                                                 
 dense_42 (Dense)            (None, 150)               22650     
                                                                 
 dropout_17 (Dropout)        (None, 150)               0         
                                                                 
 dense_43 (Dense)            (None, 100)               15100     
                                                                 
 dense_44 (Dense)            (None, 100)              

In [34]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 5s 4ms/step - loss: 0.5834 - accuracy: 0.7191
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5725 - accuracy: 0.7249
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5711 - accuracy: 0.7266
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5688 - accuracy: 0.7280
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5691 - accuracy: 0.7264
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5682 - accuracy: 0.7283
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5681 - accuracy: 0.7274
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5676 - accuracy: 0.7274
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5666 - accuracy: 0.7278
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5671 - accura

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5816 - accuracy: 0.7227 - 601ms/epoch - 2ms/step
Loss: 0.5815666317939758, Accuracy: 0.7226822376251221


In [37]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimisation.h5")
files.download('AlphabetSoupCharity_Optimisation.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>